## **Práctica 7**
## Traducción con redes neuronales

### Fecha de entrega

5 de mayo 2024 11.59 p.m.

### Actividades

- Explorar los datasets disponibles en el *Shared Task de Open Machine Translation de AmericasNLP 2021*
    - [Datasets](https://github.com/AmericasNLP/americasnlp2021/tree/main/data)
    - [Readme](https://github.com/AmericasNLP/americasnlp2021/tree/main#readme)
- Crear un modelo de traducción neuronal usando OpenNMT-py y siguiendo el pipeline visto en clase
    - 0. Obtención de datos y preprocesamiento
        - Considerar que tiene que entrenar su modelo de tokenization
    - 1. Configuración y entrenamiento del modelo
    - 2. Traducción
    - 3. Evaluación
        - Reportar BLEU
        - Reportar ChrF (medida propuesta para el shared task)
        - Más info: [evaluate.py](https://github.com/AmericasNLP/americasnlp2021/blob/main/evaluate.py)        
- Comparar resultados con [baseline](https://github.com/AmericasNLP/americasnlp2021/tree/main/baseline_system#baseline-results)
- Incluir el archivo `*.translated.desubword`

### Extra

- Investigar porque se propuso la medida ChrF en el Shared Task
    - ¿Como se diferencia de BLEU?
    - ¿Porqué es reelevante utilizar otras medidas de evaluación además de BLEU?

In [4]:
"""
Fetch data sets directly from GitHub
"""

URLs = """
https://raw.githubusercontent.com/AmericasNLP/americasnlp2021/main/data/nahuatl-spanish/dev.es
https://raw.githubusercontent.com/AmericasNLP/americasnlp2021/main/data/nahuatl-spanish/dev.nah
https://raw.githubusercontent.com/AmericasNLP/americasnlp2021/main/data/nahuatl-spanish/test.es
https://raw.githubusercontent.com/AmericasNLP/americasnlp2021/main/data/nahuatl-spanish/train.es
https://raw.githubusercontent.com/AmericasNLP/americasnlp2021/main/data/nahuatl-spanish/train.nah
"""

# Download the data

import os
import urllib.request

formatted_URLs = [ url for url in URLs.strip().split('\n') if url.strip() != '' ]
for url in formatted_URLs:
    filename = url.split('/')[-1]
    if not os.path.exists(filename):
        print(f"Downloading {filename}")
        urllib.request.urlretrieve(url, filename)
    else:
        print(f"Skipping {filename}")

print("Done!")

Skipping dev.es
Skipping dev.nah
Skipping test.es
Done!


In [ ]:
"""
Preprocessing
"""

# TODO: determine best way to preprocess the data
# read data
# tokenize
# lowercase
# remove punctuation
# remove stopwords
# remove numbers

### Exploración de datasets

### Comparación

### Evaluación

In [6]:
"""
import sacrebleu
import argparse


def calculate_score_report(sys, ref, score_only):

    chrf = sacrebleu.corpus_chrf(sys, ref)
    bleu = sacrebleu.corpus_bleu(sys, ref)

    prefix = 'BLEU = ' if score_only else ''

    print('#### Score Report ####')
    print(chrf)
    print('{}{}'.format(prefix, bleu.format(score_only=score_only)))


if __name__ == '__main__':

    parser = argparse.ArgumentParser()

    parser.add_argument('--system_output', '--sys', type=str, help='File with each line-by-line model outputs')
    parser.add_argument('--gold_reference', '--ref', type=str, help='File with corresponding line-by-line references')
    parser.add_argument('--detailed_output', action='store_const', const=True, default=False, help='(sacrebleu) Print additional BLEU information (default=False)')
    args = parser.parse_args()

    gold_lines = []
    no_translations = []
    with open(args.gold_reference, 'r') as f:
        for i, line in enumerate(f):

            if len(line.strip()) == 0:
                no_translations.append(i)
                continue

            gold_lines.append(line.strip())

    system_lines = []

    print(no_translations)

    with open(args.system_output, 'r') as f:
        for i,line in enumerate(f):

            if i in no_translations:
                continue

            system_lines.append(line.strip())


    assert len(system_lines) == len(gold_lines)

    print(len(system_lines))



    calculate_score_report(system_lines, [gold_lines], score_only=not args.detailed_output)
"""

"\nimport sacrebleu\nimport argparse\n\n\ndef calculate_score_report(sys, ref, score_only):\n\n    chrf = sacrebleu.corpus_chrf(sys, ref)\n    bleu = sacrebleu.corpus_bleu(sys, ref)\n\n    prefix = 'BLEU = ' if score_only else ''\n\n    print('#### Score Report ####')\n    print(chrf)\n    print('{}{}'.format(prefix, bleu.format(score_only=score_only)))\n\n\nif __name__ == '__main__':\n\n    parser = argparse.ArgumentParser()\n\n    parser.add_argument('--system_output', '--sys', type=str, help='File with each line-by-line model outputs')\n    parser.add_argument('--gold_reference', '--ref', type=str, help='File with corresponding line-by-line references')\n    parser.add_argument('--detailed_output', action='store_const', const=True, default=False, help='(sacrebleu) Print additional BLEU information (default=False)')\n    args = parser.parse_args()\n\n    gold_lines = []\n    no_translations = []\n    with open(args.gold_reference, 'r') as f:\n        for i, line in enumerate(f):\n\n 

In [5]:
"""
Evaluation
"""

# TODO: print evaluation metrics i.e. BLEU and ChrF

'\nEvaluation\n'

### Investigación

#### ¿Qué es ChrF y cómo funciona?

**ChrF**, o **Character n-gram F-score**, es una métrica que se utiliza para evaluar la calidad de las traducciones automáticas comparándolas con traducciones de referencia humanas. A diferencia de BLEU, que evalúa la coincidencia de n-gramas de palabras, ChrF evalúa las coincidencias a nivel de caracteres y n-gramas de caracteres. Esto implica que se concentra más en la similitud textual a un nivel más granular.

La métrica ChrF calcula tanto la precisión como la cobertura (recall) de los n-gramas de caracteres:

- **Precisión**: Proporción de n-gramas de caracteres en la traducción automática que también aparecen en la traducción de referencia.
- **Recall (Cobertura)**: Proporción de n-gramas de caracteres en la traducción de referencia que también aparecen en la traducción automática.

Estas dos medidas se combinan en una puntuación F-score, que es la media armónica de la precisión y la cobertura. La puntuación de ChrF se puede ajustar para ponderar más la precisión o la cobertura mediante un factor beta (β). Un valor común es ChrF++ (o ChrF3), que da más peso al recall para enfatizar la importancia de no perder contenido en la traducción.

#### Diferencias entre ChrF y BLEU

Mientras que BLEU es una métrica establecida y ampliamente usada en la evaluación de traducciones automáticas, presenta algunas limitaciones, especialmente en idiomas con estructuras morfológicas complejas o en textos donde la exactitud a nivel de palabra no captura completamente la calidad de la traducción. Las principales diferencias entre ChrF y BLEU son:

- **Granularidad**: ChrF evalúa a nivel de caracteres, lo que la hace más sensible a las variaciones morfológicas y ortográficas entre la traducción y la referencia.
- **Robustez en idiomas morfológicamente ricos**: Para idiomas como el árabe, turco o finés, donde las palabras pueden alterar significativamente su forma debido a la conjugación o la formación de palabras, ChrF puede ofrecer una evaluación más precisa que BLEU.
- **Menos sensibilidad a los errores de tokenización**: Dado que BLEU depende de una tokenización precisa, cualquier error en este proceso puede afectar significativamente su puntuación. ChrF, al basarse en caracteres, es menos susceptible a estos problemas.

#### Importancia de utilizar múltiples métricas de evaluación

Depender exclusivamente de BLEU para evaluar sistemas de traducción automática puede llevar a una comprensión incompleta de su rendimiento. Cada métrica tiene sus fortalezas y debilidades, y diferentes métricas pueden resaltar diferentes aspectos de la calidad de la traducción. Al utilizar ChrF junto con BLEU, los evaluadores pueden obtener una visión más completa y matizada del desempeño de un sistema de traducción, especialmente en términos de cómo maneja las diferencias lingüísticas y culturales entre los idiomas de origen y destino.